In [93]:
import pandas as pd 
import numpy as np
from nltk import WordPunctTokenizer
from nltk import TreebankWordTokenizer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from pprint import pprint
import os 
import time

In [14]:
# open dataset and preprocess
data = pd.read_csv('..\\sarcasm_dataset_clean.csv')          # import the data
labels = np.array(data['label'].tolist())                    # extracts the labels as a list 
text = data['text'].tolist()                                 # extracts the text to be processed as a list

In [15]:
# tok = TreebankWordTokenizer()
# tokenizer=tok.tokenize, use this if the results are poor
minfreq = 3                                              # Used to prune those terms (uni-|bi-|tri- grams whose frequency is less than this)

In [4]:
print(text[0])

Do you remember me telling you we are practicing non-verbal spells, Potter?" "Yes," said Harry stiffly. "Yes, sir." "There's no need to call me "sir" Professor." The words had escaped him before he knew what he was saying.  '


In [120]:
def process_data(text,labels,n_gram,tokenizer=None,min_document_frequency=2,minfreq=3):
    '''
        Tokenizes the dataset and return the term-frequency matrix of the frequent n-grams 
        
        Parameters
        -----------
        text : list
            List of strings containing the text to be processed.
        n_gram: int
            The size of the n-gram to be returned
        tokenizer:
            String tokenizer to be used
        min_document_frequency: 
            The minimum document frequency to be used to prune n-grams
        minfreq: 
            The minimum frequency to be used to prune n-grams
            
        Returns
        -------
        dataframe : Pandas DataFrame
            The text converted as a term-frequency matrix
        n_grams : list
            The list of n-grams that were generated
        weights: tuple 
            p_value , chi2 value 
    '''
    vectorizer = CountVectorizer(tokenizer=tokenizer,lowercase=False,\
                                 ngram_range=(n_gram,n_gram),\
                                 min_df=min_document_frequency,\
                                 stop_words='english',\
                                 token_pattern='\\w+')
    
    processed_data = (vectorizer.fit_transform(text))                      
    processed_data = processed_data.toarray()                              
    
    n_grams = vectorizer.get_feature_names()                               
    
    counts = np.sum(processed_data,axis=0)                                 
    indices_to_keep = (np.argwhere(counts > minfreq)).flatten()            
    processed_data = processed_data[:,indices_to_keep] 
#     print(type(indices_to_keep))
    n_grams = [n_grams[i]  for i in indices_to_keep]
    dataframe = pd.DataFrame(processed_data)
    
    weights = chi2(processed_data,labels)
    
    return dataframe,n_grams,weights

In [83]:
def store(file_name,data,labels):
#     <line> .=. <target> <feature>:<value> <feature>:<value> ... <feature>:<value> # <info>
#     <target> .=. {+1,-1}
#     <feature> .=. <integer>
#     <value> .=. <float>
#     <info> .=. <string>
    labels = 2*labels - 1
    with open(file_name+'.dat','w') as f :
        final_string = ""
        for idx,row in data.iterrows():
            string = ''
            string += '+1 ' if labels[idx] ==1 else '-1 '
            column_indices = data.columns
            for index in column_indices:
                if row[index] == 0:
                    continue
                string += str((index+1))+':'+str(row[index])+' '
            string += '\n'
            final_string += string
        f.write(final_string[:-1])
        print('Finished Storing')
        return final_string

In [134]:
unigrams_matrix,unigrams,(unigram_chi,unigram_p) = process_data(text,labels,1)
unigram_features = unigrams_matrix*unigram_chi
print(unigrams_matrix.shape)

(3756, 3469)


In [135]:
bigrams_matrix,bigrams,(bigram_chi,bigram_p) = process_data(text,labels,2)
bigram_features = bigrams_matrix*bigram_chi
print(bigrams_matrix.shape)

(3756, 594)


In [136]:
trigrams_matrix,trigrams,(trigram_chi,trigram_p) = process_data(text,labels,3)
trigram_features = trigrams_matrix*trigram_chi
print(trigrams_matrix.shape)
# print(trigrams)
# print(trigrams_matrix)

(3756, 63)


In [137]:
final_string = store('train_data',trigrams_matrix,labels)
from sklearn.model_selection import KFold
# print(final_string)

Finished Storing


In [138]:
def shuffle(df):
    '''
        Shuffles pandas Dataframe
        
        Parameters
        -----------
        df : pd.DataFrame
            Dataframe to be shuffled
    
                
        Returns
        -------
        df_copy : Pandas DataFrame
            Shuffled Dataframe copy is returned    
    '''
    df_copy = df.copy()
    df_copy = df_copy.sample(frac=1).reset_index(drop=True)
    return df_copy

def customKfold(data, labels, n_folds=5, random_seed=99):
    # Note: Will not shuffle data - Shuffle before and pass
    '''
        Stores train and test .dat files for each fold
        
        Parameters
        -----------
        data : pd.DataFrame
            Feature vectors
        labels: list
            True Labels
        n_folds:
            Number of folds required
        random_seed: 
            Random seed to generate the split
    '''
    from sklearn.model_selection import KFold
    kfold = KFold(n_folds, True, random_seed)
    iteration = 1
    for train, test in kfold.split(data):
        print(iteration)
        X_train, X_test, y_train, y_test = np.array(data.iloc[train]), np.array(data.iloc[test]), labels[train], labels[test]
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        store('data/train_'+str(iteration), X_train, y_train)
        store('data/test_'+str(iteration), X_test, y_test)
        iteration += 1

In [139]:
lieb_f = pd.concat([unigram_features, bigram_features, trigram_features], axis = 1)

In [ ]:
features = shuffle(lieb_f)
start_time = time.time()
customKfold(features, labels)
end_time = time.time()-start_time
print(end_time)

1
Finished Storing
Finished Storing
2
